<a href="https://colab.research.google.com/github/raghavgate/Machine-Learning-and-Artificial-Intelligence-Projects/blob/main/Agentic_Workflow_for_News_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import pipeline
from typing import Dict


In [ ]:
# Load a free Hugging Face model (FLAN-T5-XL works well for reasoning)
generator = pipeline("text2text-generation", model="google/flan-t5-xl")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.45G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
# Agent state type
AgentState = Dict[str, str]

In [ ]:
# Agent Functions
def news_ingest_function(state: AgentState) -> AgentState:
    news_text = "Tesla stock plunges as Elon Musk resigns as CEO"
    state["news"] = news_text
    return state

def summarize_news(state: AgentState) -> AgentState:
    prompt = f"Summarize the financial news and extract company names and tickers: {state['news']}"
    response = generator(prompt, max_new_tokens=100)[0]['generated_text']
    state["summary"] = response
    return state

def analyze_sentiment(state: AgentState) -> AgentState:
    prompt = f"What is the sentiment of this news? (Positive, Neutral, Negative). Reason: {state['summary']}"
    response = generator(prompt, max_new_tokens=100)[0]['generated_text']
    state["sentiment"] = response
    return state

def evaluate_risk(state: AgentState) -> AgentState:
    prompt = f"Is this event High, Medium, or Low risk for the stock price? Reason: {state['summary']}"
    response = generator(prompt, max_new_tokens=100)[0]['generated_text']
    state["risk"] = response
    return state

def decide_action(state: AgentState) -> AgentState:
    sentiment = state.get("sentiment", "")
    risk = state.get("risk", "")
    decision = "Hold"
    if "Negative" in sentiment and "High" in risk:
        decision = "Sell or Alert"
    elif "Positive" in sentiment and "High" in risk:
        decision = "Buy or Alert"
    state["decision"] = decision
    return state

def take_action(state: AgentState) -> AgentState:
    print(f"[ACTION]: {state['decision']} based on news: {state['news']}")
    return state

In [ ]:
# Full Pipeline
def run_agent():
    state: AgentState = {
        "news": "",
        "summary": "",
        "sentiment": "",
        "risk": "",
        "decision": ""
    }

    state = news_ingest_function(state)
    state = summarize_news(state)
    state = analyze_sentiment(state)
    state = evaluate_risk(state)
    state = decide_action(state)
    state = take_action(state)

    # Print final state
    print("\n--- Final Agent State ---")
    for k, v in state.items():
        print(f"{k.capitalize()}: {v.strip()}")

run_agent()

[ACTION]: Sell or Alert based on news: Tesla stock plunges as Elon Musk resigns as CEO

--- Final Agent State ---
News: Tesla stock plunges as Elon Musk resigns as CEO
Summary: Tesla stock plunges as Elon Musk resigns as CEO
Sentiment: Negative
Risk: High
Decision: Sell or Alert
